# Data Node追加操作
----
新しいData&nbsp;Nodeを クラスタに追加します。

**<font color="red">このデモ環境では実施できません。クラスタを運用した場合のイメージとしてお読みください。</font>**

## 準備
本章のコマンドを実行するための設定を行います。

セットアップ済みの環境のうちコマンドを送る先であるCoordinating(Client) Nodeのホストアドレスを、次のセルに記入して実行し、保存してください。

In [1]:
%env ES_HOST=XXX.XXX.XXX.233:9200

env: ES_HOST=XXX.XXX.XXX.233:9200


## サーバの追加生成
Data Nodeを生成する先のサーバを生成します。

[01_02_Accommodation_AWS.ipynb](01_02_Accommodation_AWS.ipynb)を開きます。  

- 追加したいサーバのスペックを入力してください。
- instance_count は全台数でなく、追加したい台数を入力します。
- Notebookを最後まで走行させてください。追加したサーバのホスト情報を得ます。

## インベントリへホストを追加
追加したホスト情報を既存のインベントリに追記します。

バックアップ用ディレクトリを生成します。

In [4]:
from datetime import datetime as dt
import os
bak_dir = './bak_' + dt.now().strftime('%Y%m%d')
if(not os.path.isdir(bak_dir)) : os.mkdir(bak_dir)
print(bak_dir)

./bak_20170322


ファイルを生成したディレクトリにコピーします。

In [5]:
!cp ./hosts {bak_dir}/hosts

hostsファイルを次のリンクから編集してください。

- **[hosts](../edit/hosts)**

変更結果を既存の内容（バックアップディレクトリに退避しているもの）と比較するには、次のセルを実行してください。

In [6]:
!diff -ur {bak_dir}/hosts hosts

--- ./bak_20170322/hosts	2017-03-22 14:58:53.348890250 +0900
+++ hosts	2017-03-22 14:59:06.044996937 +0900
@@ -14,8 +14,6 @@
 
 [ingest-nodes]
 ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com
-ec2-54-249-8-131.ap-northeast-1.compute.amazonaws.com
-ec2-54-249-98-66.ap-northeast-1.compute.amazonaws.com
 
 [logstash-server]
 ec2-54-238-216-212.ap-northeast-1.compute.amazonaws.com


## 追加したホストへのインストール
追加したホストに、Elasticsearchをインストールし、設定します。

[01_04_Install.ipynb](01_04_Install.ipynb)を開き、走行させてください。  

## クラスタの状態確認
クラスタに追加したData Nodeが参加できているか確認します。

（[01_04_Install.ipynb](01_04_Install.ipynb)でも実施していますが）クラスタのData Node数が増えていることを確認してください。  

Client Nodeのアドレスか、それが無い場合はData Nodeのうちの1つのアドレスを設定してください。 

次のコマンドを実施し、クラスタの状態を確認します。

In [3]:
!curl -XGET http://$ES_HOST/_cluster/health?pretty

{
  "cluster_name" : "es-cluster",
  "status" : "green",
  "timed_out" : false,
  "number_of_nodes" : 3,
  "number_of_data_nodes" : 3,
  "active_primary_shards" : 39,
  "active_shards" : 78,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 0,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 100.0
}


"number_of_data_nodes"が、増えた結果のData Node数と一致しているか確認してください。

## Data Node追加・障害復旧時の注意点
Data Nodeを追加したり、一度障害が発生してクラスタから抜けたData Nodeを復旧させる際の注意点について説明します。

1. データが入った状態のNodeを追加した場合、ドキュメントIDが重複してしまう可能性があります。  
    その場合追加された方が無視されるので結果としてデータが入っていないように見えてしまいます。  
    Data Nodeを追加する場合はデータが入っていないものを追加するようにしてください。
1. Data Nodeが追加、または削除された場合、クラスタのデータが再構成されて均等に分散されます。  
    この処理にはCPU,メモリを消費するためその期間は性能が落ちます。Task Management APIを使用して実行中のイベントを確認することができます。  
    Task APIについてはElasticsearch Referenceの[Task Management API](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/tasks.html)を参照してください。
    
    ノード内の実行中のイベントを確認するコマンドは次の通りです。

In [ ]:
!curl -XGET http://$ES_HOST/_tasks?group_by=parents&pretty